In [2]:
import torch
import torch.nn as nn
import math

# 位置编码模块
class PositionalEncoding(nn.Module):
    # 传入最大位置长度max_pos和词向量的维度embed_dim
    def __init__(self, max_pos, embed_dim):
        super(PositionalEncoding, self).__init__()
        # 初始化位置编码矩阵，全部为零
        PE = torch.zeros(max_pos, embed_dim)  # 保存位置编码的数组
        # 生成从0到max_pos-1的位置数组pos
        pos = torch.arange(0, max_pos).unsqueeze(1).float()

        # 从0开始，生成间隔为2的序列，对应公式中的2i
        multi_term = torch.arange(0, embed_dim, 2).float()
        # 计算公式中pos对应的系数部分
        # 这里计算的是e^(2i * (-log(10000/d)))
        # 它从数学计算上等价于1 / (10000^(2i/d))
        multi_term = torch.exp(multi_term * (-math.log(10000.0) / embed_dim))
        # 使用正弦函数sin和余弦函数cos，生成位置编码数组PE
        PE[:, 0::2] = torch.sin(pos * multi_term)
        PE[:, 1::2] = torch.cos(pos * multi_term)

        # 将数组PE注册为一个不需要梯度更新的缓存数组
        # 相当于将位置信息保存在了一个常量数组中
        self.register_buffer('PE', PE.unsqueeze(0))

    # 前向传播函数，函数传入输入数据x
    def forward(self, x):
        # 将x加上位置信息PE
        return x + self.PE[:, :x.size(1)].clone().detach()

if __name__ == "__main__":
    max_pos = 10  # 最大序列长度
    embed_dim = 4  # 词向量维度

    # 定义位置编码模型
    model = PositionalEncoding(max_pos, embed_dim)

    # 输入数据，代表了2个样本，每个样本长度是5
    x = torch.zeros(2, 5, embed_dim)

    # 将x传入模型model，计算添加位置信息信息的结果output
    output = model(x)

    print("x:")
    print(x)
    print("PE:")
    print(model.PE)
    print("output:")
    print(output)

x:
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
PE:
tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000],
         [ 0.8415,  0.5403,  0.0100,  0.9999],
         [ 0.9093, -0.4161,  0.0200,  0.9998],
         [ 0.1411, -0.9900,  0.0300,  0.9996],
         [-0.7568, -0.6536,  0.0400,  0.9992],
         [-0.9589,  0.2837,  0.0500,  0.9988],
         [-0.2794,  0.9602,  0.0600,  0.9982],
         [ 0.6570,  0.7539,  0.0699,  0.9976],
         [ 0.9894, -0.1455,  0.0799,  0.9968],
         [ 0.4121, -0.9111,  0.0899,  0.9960]]])
output:
tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000],
         [ 0.8415,  0.5403,  0.0100,  0.9999],
         [ 0.9093, -0.4161,  0.0200,  0.9998],
         [ 0.1411, -0.9900,  0.0300,  0.9996],
         [-0.7568, -0.6536,  0.0400,  0.9992]],

  